<a href="https://colab.research.google.com/github/paulinepeh/WhoDini/blob/main/yolo11_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics kaggle


In [ ]:
import kagglehub
path = kagglehub.dataset_download("lylmsc/wider-face-for-yolo-training")


In [ ]:
import kagglehub
path = kagglehub.dataset_download("lylmsc/wider-face-for-yolo-training")
print(path)


In [ ]:
DATA_PATH = path
!find $DATA_PATH -maxdepth 3 -type d


In [ ]:
import os, random, shutil

# 1)  dataset stored in Kaggle
DATA_PATH = "/root/.cache/kagglehub/datasets/lylmsc/wider-face-for-yolo-training/versions/1"

# 2) Where we will create the YOLO-ready train/val folders
OUT = "/content/wider_face_yolo"

# Make folders
for folder in ["images/train", "images/val", "labels/train", "labels/val"]:
    os.makedirs(os.path.join(OUT, folder), exist_ok=True)

# Get all image files
images = [f for f in os.listdir(DATA_PATH + "/images") if f.endswith(".jpg")]
random.shuffle(images)

# 90% train, 10% val
cut = int(0.9 * len(images))
train = images[:cut]
val = images[cut:]

def copy_pairs(file_list, split_name):
    copied = 0
    for img in file_list:
        label = img.replace(".jpg", ".txt")

        img_src = DATA_PATH + "/images/" + img
        lbl_src = DATA_PATH + "/labels/" + label

        # Only copy if label exists
        if os.path.exists(lbl_src):
            shutil.copy(img_src, OUT + f"/images/{split_name}/" + img)
            shutil.copy(lbl_src, OUT + f"/labels/{split_name}/" + label)
            copied += 1

    print(split_name, "copied:", copied)

copy_pairs(train, "train")
copy_pairs(val, "val")


In [ ]:
import os, cv2
import matplotlib.pyplot as plt

img_dir = "/content/wider_face_yolo/images/train"
lbl_dir = "/content/wider_face_yolo/labels/train"

img_name = os.listdir(img_dir)[0]  # pick  first image
img_path = img_dir + "/" + img_name
lbl_path = lbl_dir + "/" + img_name.replace(".jpg", ".txt")

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w = img.shape[:2]

with open(lbl_path) as f:
    for line in f:
        cls, x, y, bw, bh = map(float, line.split())

        # Convert YOLO normalized coords -> pixel coords
        x, y, bw, bh = x*w, y*h, bw*w, bh*h
        x1, y1 = int(x - bw/2), int(y - bh/2)
        x2, y2 = int(x + bw/2), int(y + bh/2)

        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)

plt.figure(figsize=(6,6))
plt.imshow(img)
plt.axis("off")
plt.show()


In [ ]:
open("/content/wider_face.yaml", "w").write("""
path: /content/wider_face_yolo
train: images/train
val: images/val

names:
  0: face
""")


In [ ]:
!yolo detect train model=yolo11n.pt data=/content/wider_face.yaml epochs=3 imgsz=640 batch=8


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.0 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/wider_face.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, kera